In [1]:
from BCBio import GFF
import operator
from Bio.SeqRecord import SeqRecord
import glob, os
from Bio import SeqIO
import time
import pysam

import gffutils
import pyfaidx
from pysam import VariantFile
from Bio.Seq import Seq

In [2]:
sourcebase= '/mnt/test_data/hepavac34/rna_benign/spladdrout'
#sourcebase = '/tmp/'
reffile = '/mnt/test_data/refs/GRCh37.primary_assembly.genome.fa'
fileending = 'gff3'
vcffile= '/tmp/vcf.vcf.gz' #must be indexed!

#db = gffutils.create_db('/tmp/ensg228794.gff', ':memory:')
fasta = pyfaidx.Fasta('/mnt/test_data/refs/GRCh37.primary_assembly.genome.fa')
vcffile='/tmp/vcf.vcf.gz'

outfile = '/tmp/predicted_exons.fa'
outfile2 = '/tmp/predicted_genes.fa'  
vcfrecords=VariantFile(vcffile)


In [3]:
#Getting all gff-files in base directory
def get_gff_files(sourcebase):
    print "Checking for GFF Files in directory..."
    gfffiles = []
    os.chdir(sourcebase)
    for file in glob.glob("*.%s" %fileending):
        gfffiles.append(file)
    print "Found %i file(s) in %s ending with %s." %(len(gfffiles), sourcebase, fileending)
    return gfffiles

In [128]:
def get_gene_sequences(gene,db):
    sequences = []
    geneseq = []
    num_mod=0
    #print gene
    for i in db.children(gene,featuretype='exon'):
        events=check_feature_for_vcfevent(i)
        if len(events) == 0:
            rec = i.sequence(fasta)#.translate(to_stop=True)
            sequences.append(SeqRecord(Seq(rec),i.id,i.attributes['Parent'][0],""))
            geneseq.append(rec)
        else:
            num_mod+=1
        #    sequences.append(Seq.translate(i.sequence(fasta),to_stop=True))
            rec = get_modified_sequence(i.sequence(fasta),events,i.start)
            sequences.append(SeqRecord(Seq(rec),i.id,i.attributes['Parent'][0],""))
            geneseq.append(rec)
            #sequences.append(get_modified_sequence(i.sequence(fasta),events,i.start))
            #return get_modified_sequence(i.sequence(fasta),events)
        #if i.id!='exon_7569':
        #    print "this is the record"
        #    print sequences[-1]
        
    return sequences,[SeqRecord(Seq(''.join(geneseq)),gene.id,"")], num_mod 

In [129]:
def check_feature_for_vcfevent(feature):
    vcfevents = []
    if feature.chrom not in vcfrecords.header.contigs:
        print "Chrom not found"
        return vcfevents
    for rec in vcfrecords.fetch(feature.chrom,feature.start,feature.stop):
        vcfevents.append(rec)
       #a=rec
    if len(vcfevents) == 0:
        return vcfevents
        print "no events in range detected (%s - %i on %i)"%(feature.chrom,feature.start,feature.stop)
    else:
        print "%i events in range detected (%s - %i on %i)"%(len(vcfevents),feature.chrom,feature.start,feature.stop)
        
    return vcfevents

In [130]:
def get_modified_sequence(in_seq,events,offset):
    print "Personalisizing sequence len:%i with %i events"%(len(in_seq),len(events))
    lengthchange=0
    for event in events:
        eventpos=event.start-offset
        print "Eventstart %i, offset: %i -> pos:%i"%(event.start,offset,eventpos)
        print "Char at  %i: %s (region +-3: %s)" %(eventpos,in_seq[eventpos],in_seq[eventpos-3:eventpos+3])
        alleles=event.alleles
        l = list(in_seq)
        
        if is_snp(alleles):
          #  print "Found SNP substituting %s with %s" %(l[eventpos],alleles[1])
            if l[eventpos]!= alleles[0]:
                print "warning SNP missmatch"
            l[eventpos]=alleles[1]
        if is_del(alleles):
            l[eventpos]=alleles[1] #TODO falsch!
            lengthchange-=len(alleles[1])-len(alleles[0])
            print "Found deletion: deleting %s from sequences"%alleles[0]
            print alleles
            print event
        if is_insert(alleles):
            l.insert(eventpos,alleles[1])
            lengthchange+=len(alleles[0])-len(alleles[1])
            print "found insert: inserting %s into sequence at pos %i"%alleles[1],eventpos
            print event
    new_seq=''.join(l)
    if (len(in_seq)-len(new_seq)) != lengthchange:
        print "Warning: length changed %i (should be %i)" %(len(in_seq)-len(new_seq),lengthchange)
    return new_seq 

In [131]:
def is_snp(e):
    if len(e[0]) == 1 and len(e[1]) == 1:
        return True
    return False

def is_insert(e):
    if len(e[0])<len(e[1]):
        return True
    return False

def is_del(e):
    if len(e[0])>len(e[1]):
        return True
    return False

In [137]:
def run():
    gfffiles = get_gff_files(sourcebase)
    
    for infile in gfffiles:
        total_genes=0
        total_exons=0
        total_genes_wrote=0
        total_exons_wrote=0
        total_mod=0
        db = gffutils.create_db(infile, ':memory:')
        for gene in db.features_of_type('gene'):
            total_genes+=1
            seqs,geneseq,num_mod=get_gene_sequences(gene,db)
            total_exons+=len(seqs)
            total_mod+=num_mod
            #a=seqs
           # return seqs
            #return geneseq
            total_exons_wrote+=write_records(translate_records(seqs),'exons')
            total_genes_wrote+=write_records(translate_records(geneseq),'genes')
        print "processed (and wrote): %s with %i(%i) genes and %i(%i) exons (%i modified))" %(infile,total_genes,total_genes_wrote,total_exons,total_exons_wrote,total_mod)
        
    #ref_recs = load_reffile(reffile)
    
     #   records, records_until_stop = get_records(infile,ref_recs)
      #  write_records(records,records_until_stop,os.path.basename(infile))
        

In [140]:
def write_records(records,type):
    #translate records
    
    if type == 'exons':
        with open(outfile, "a") as out_handle:
            #print(records[1])
            return SeqIO.write(records, out_handle, "fasta")
    if type == 'genes':
        with open(outfile2, "a") as out_handle:
            #print(records[1])
            return SeqIO.write(records, out_handle, "fasta")

In [141]:
a=run()

Checking for GFF Files in directory...
Found 5 file(s) in /mnt/test_data/hepavac34/rna_benign/spladdrout ending with gff3.
Chrom not found
Chrom not found
Chrom not found
1 events in range detected (chr17 - 26873725 on 26874867)
Personalisizing sequence len:1143 with 1 events
Eventstart 26874689, offset: 26873725 -> pos:964
Char at  964: G (region +-3: CACGGT)
warning SNP missmatch
1 events in range detected (chr4 - 155491571 on 155491855)
Personalisizing sequence len:285 with 1 events
Eventstart 155491783, offset: 155491571 -> pos:212
Char at  212: C (region +-3: GGCCCT)
1 events in range detected (chr4 - 155491571 on 155491951)
Personalisizing sequence len:381 with 1 events
Eventstart 155491783, offset: 155491571 -> pos:212
Char at  212: C (region +-3: GGCCCT)
1 events in range detected (chr6 - 109767339 on 109767560)
Personalisizing sequence len:222 with 1 events
Eventstart 109767448, offset: 109767339 -> pos:109
Char at  109: T (region +-3: CCCTGA)
1 events in range detected (chr6 

processed (and wrote): merge_graphs_alt_3prime_C3.confirmed.gff3 with 7030(21090) genes and 28120(84360) exons (12 modified))
1 events in range detected (chr14 - 105221967 on 105222161)
Personalisizing sequence len:195 with 1 events
Eventstart 105221961, offset: 105221967 -> pos:-6
Char at  -6: A (region +-3: CCGAAG)
Found deletion: deleting CCCAGAGAAGACCAA from sequences
('CCCAGAGAAGACCAA', 'C')
chr14	105221962	.	CCCAGAGAAGACCAA	C	.	PASS	ANN=C|frameshift_variant&splice_acceptor_variant&splice_region_variant&intron_variant|HIGH|SIVA1|ENSG00000184990|transcript|ENST00000556195|protein_coding|2/4|c.170-4_179delCCAGAGAAGACCAA|p.Glu57fs||170/579|57/192||WARNING_TRANSCRIPT_NO_START_CODON,C|frameshift_variant&splice_acceptor_variant&splice_region_variant&intron_variant|HIGH|SIVA1|ENSG00000184990|transcript|ENST00000329967|protein_coding|2/4|c.119-4_128delCCAGAGAAGACCAA|p.Glu40fs||119/528|40/175||,C|frameshift_variant&splice_acceptor_variant&splice_region_variant&intron_variant|HIGH|SIVA1|ENS

In [116]:
def translate_records(records):
    new_records = []
    for rec in records:
        for i in range(0,3):
            new_records.append(SeqRecord(rec.seq[i:].translate(to_stop=True),"%s %s TL-Windows: %i "%(rec.name,rec.id,i),rec.name,""))
    return new_records

In [135]:
b=translate_records(a)

In [126]:
a

SeqRecord(seq=Seq('AGTGAATATACGTGCCCTGTTTGTTGAATCTACTCATCCTTAAAGGCTGAATGC...AAG', Alphabet()), id='intron_retention.6', name='', description='<unknown description>', dbxrefs=[])

In [136]:
write_records(b,'genes')

3